### Call External tools via AI Agent 
- Retrieve data from external api, query database, execute code, search web, control a computer

In [ ]:
from anthropic import Anthropic
from dotenv import load_dotenv
import os
load_dotenv()
MODEL_NAME= "claude-3-5-sonnet-20241022"
client = Anthropic()

#### Define a system prompt 
- This needs to be added to the beginning of the message to be sent to Claude

In [2]:
system_prompt = """You are a customer support bot for a online event registration platform. Your job is to help the users with their registration status by looking up via their name or email or state. You have access to set of tools, but use them only when you need to find some specific data based on the user input. Provide concise responses and do not provide any data outside the scope of the user query. Do not call tools if the user has not provided any specific input data. 

In each conversational turn, you will begin by thinking about your response. Once you are done, you will write a user facing response. It is important to place all your user facing responses in <reply></reply> XML tags that would make for the application layer to easily parse.  """

#### Read a csv file for data and use this as data source

In [3]:
import pandas as pd
event_df = pd.read_csv('./local_file.csv')
def get_data_by_state(state):
    data_state = event_df.query("State == @state")
    return data_state

def get_data_by_fname(fname):
    data_fname = event_df.query("First == @fname")
    return data_fname

def get_data_by_lname(lname):
    data_lname = event_df.query("Last == @lname")
    return data_lname

def get_data_by_email(email):
    data_email = event_df.query("Email == @email")
    return data_email

def get_data_by_hotel(hotel):
    data_hotel = event_df.query("Hotel == @hotel")
    return data_hotel

def get_data_by_food(food):
    data_food = event_df.query("Food == @food")
    return data_food


#### Define tool schema for the Agent

In [4]:
tools = [
    {
        "name": "get_data_by_state",
        "description": "Get data by input state value",
        "input_schema":{
            "type": "object",
            "properties": {
                "state": {
                    "type": "string",
                    "description": "Input value of state name"
                }
            },
            "required": ["state"]
        }
    }, 
    {
        "name": "get_data_by_fname",
        "description": "Get data from input first name value",
        "input_schema":{
            "type": "object",
            "properties": {
                "fname": {
                    "type": "string",
                    "description": "Input value of first name"
                }
            },
            "required": ["fname"]
        }
    }, 
    {
        "name": "get_data_by_lname",
        "description": "Get data from input last name value",
        "input_schema":{
            "type": "object",
            "properties": {
                "lname": {
                    "type": "string",
                    "description": "Input value of last name"
                }
            },
            "required": ["lname"]
        }
    }, 
    {
        "name": "get_data_by_email",
        "description": "Get data from input email value",
        "input_schema":{
            "type": "object",
            "properties": {
                "email": {
                    "type": "string",
                    "description": "Input value of email"
                }
            },
            "required": ["email"]
        }
    }, 
    {
        "name": "get_data_by_hotel",
        "description": "Get data from input hotel value",
        "input_schema":{
            "type": "object",
            "properties": {
                "hotel": {
                    "type": "string",
                    "description": "Input value of hotel"
                }
            },
            "required": ["hotel"]
        }
    }, 
    {
        "name": "get_data_by_food",
        "description": "Get data from input food value",
        "input_schema":{
            "type": "object",
            "properties": {
                "food": {
                    "type": "string",
                    "description": "Input value of food"
                }
            },
            "required": ["food"]
        }
    }      
]

#### Define method to call tool once Assistant determines to call tools

In [5]:
def process_tool_call(tool_name, input_data):
    if tool_name == "get_data_by_state":
        return get_data_by_state(input_data['state'])
    elif tool_name == "get_data_by_fname":
        return get_data_by_fname(input_data['fname'])
    elif tool_name == "get_data_by_lname":
        return get_data_by_lname(input_data['lname'])
    elif tool_name == "get_data_by_email":
        return get_data_by_email(input_data['email'])
    elif tool_name == "get_data_by_hotel":
        return get_data_by_hotel(input_data['hotel'])
    elif tool_name == "get_data_by_food":
        return get_data_by_food(input_data['food'])

### Define a chat flow (main method that gets invoked)
- type quit as user input to stop

In [33]:
def claude_chat():
    user_message = input("\nUser:")
    system_message = """You are a customer support bot for a online event registration platform. Your job is to help the users with their registration status by looking up via their name or email or state. You have access to set of tools, but use them only when you need to find some specific data based on the user input. Provide concise responses and do not provide any data outside the scope of the user query. Do not call tools if the user has not provided any specific input data. 

    In each conversational turn, you will begin by thinking about your response. Once you are done, you will write a user facing response. It is important to place all your user facing responses in <reply></reply> XML tags that would make for the application layer to easily parse.  """
    messages = [
        {
            "role": "user",
            "content": user_message,
        }
    ]
    while True:
        if user_message.lower() in ["exit","quit","end chat"] :
            break
        
        # Send response to Claude
        response = client.messages.create(
            model=MODEL_NAME,
            messages=messages,
            system=system_message,
            tools=tools,
            max_tokens=1000,
        )

        # Append response to messages
        messages.append({
            "role": "assistant",
            "content": response.content[0].text,
        })

        # If assistant stop reason tool usage, then apply tool usage
        if response.stop_reason == "tool_use":
            tool_use = response.content[-1]
            tool_name = tool_use.name
            tool_input = tool_use.input
            print(f"Claude wants to use tool: {tool_name} with input: {tool_input}")
            # Call tool function
            tool_output = process_tool_call(tool_name, tool_input)

            # Append tool output to messages    
            messages.append({
                "role": "assistant",
                "content": tool_output.to_string(index=False),
            })
        else:
            # Claude does not want to use any tools, just print the response message
            print(f"Claude: {response.content[0].text}")



#### Invole Chat Client

In [ ]:
claude_chat()